# Libraries

In [7]:
import pandas as pd
import numpy as np

from collections import defaultdict
import re
import nltk

from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer

from sklearn.metrics.pairwise import cosine_similarity
from heapq import heappush, heappop, nlargest,heapify

import enchant 
import pickle

# Step 1: Data

In [2]:
airbnb_data=pd.read_csv("Airbnb_Texas_Rentals.csv",usecols=['average_rate_per_night', 'bedrooms_count', 'city',
       'date_of_listing', 'description', 'latitude', 'longitude', 'title','url'],parse_dates=['date_of_listing'])

In [3]:
airbnb_data.columns

Index(['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing',
       'description', 'latitude', 'longitude', 'title', 'url'],
      dtype='object')

# Step 2: Create documents

In [4]:
airbnb_data.head()

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$27,2,Humble,2016-05-01,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,$149,4,San Antonio,2010-11-01,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,$59,1,Houston,2017-01-01,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...
3,$60,1,Bryan,2016-02-01,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...
4,$75,2,Fort Worth,2017-02-01,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...


In [5]:
airbnb_data.shape

(18259, 9)

# Clean data

In [6]:
# Check null values of the dataset
airbnb_data.isnull().sum()
#average_rate_per_night -> replace NAN with 0, convert to int
#bedrooms_count -> There are only 3 records so we decided to replace NAN with a category based on the desciption if it's possible. 
#description, latitude, longitude, title -> replace NAN to 'Unknown'

average_rate_per_night    28
bedrooms_count             3
city                       0
date_of_listing            0
description                2
latitude                  34
longitude                 34
title                      3
url                        0
dtype: int64

In [7]:
airbnb_data.dtypes

average_rate_per_night            object
bedrooms_count                    object
city                              object
date_of_listing           datetime64[ns]
description                       object
latitude                         float64
longitude                        float64
title                             object
url                               object
dtype: object

In [8]:
def clean(airbnb_data):
    """
    Method that removes nan values and imputes them
    
    Input: dataframe
    Output: cleaned dataframe
    
    """
    #replace NAN with 0
    airbnb_data.average_rate_per_night.replace(np.nan, '$0',inplace=True)
    #convert to int and remove $
    airbnb_data.average_rate_per_night=airbnb_data.average_rate_per_night.replace('[\$]', '', regex=True).astype(int)

    #replace NAN with'unknown'

    airbnb_data.description.replace(np.nan,'unknown',inplace=True)
    airbnb_data.title.replace(np.nan,'unknown',inplace=True)

    airbnb_data.latitude.replace(np.nan,'unknown',inplace=True)
    airbnb_data.longitude.replace(np.nan,'unknown',inplace=True)

    #check where bedrooms_count doesn't have a value and save indexes of those records to a list
    null_value_idx=airbnb_data[airbnb_data.bedrooms_count.isnull()].index
    #if the word studio is mentioned in description then it is a studio otherwise 'unknown'
    for idx in null_value_idx:
        if 'studio' in airbnb_data.iloc[idx].description.split():
            airbnb_data.bedrooms_count[idx]='Studio'
        else:
            airbnb_data.bedrooms_count[idx]='unknown'
        
    return airbnb_data

In [9]:
airbnb_data=clean(airbnb_data)
airbnb_data.isnull().sum()

C:\Users\Dusica\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dusica\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


average_rate_per_night    0
bedrooms_count            0
city                      0
date_of_listing           0
description               0
latitude                  0
longitude                 0
title                     0
url                       0
dtype: int64

In [10]:
airbnb_data.shape

(18259, 9)

In [11]:
def create_tsv_documents(airbnb_data):
    """
    Method that creates different .tsv files for each record in the airbnb_data 
    
    Input: dataframe
    """   
    #clean data
    airbnb_data=clean(airbnb_data)
    
    #for each index make a dataframe of airbnb_data and store it into new tsv file
    for i in airbnb_data.index:
        pd.DataFrame(airbnb_data.loc[i]).transpose().to_csv('data/doc_'+str(i)+'.tsv',sep='\t')

#method is run only once at the beginning to make separate .tsv files
create_tsv_documents(airbnb_data)

# Preprocessing

1) Removing stop words

2) Removing punctuation

3) Stemming

##### remove non english words and words Giulia chooses (room, price, airbnb) MOST often ones_?
##### should we remove numbers__?

# 3.1) Conjunctive query

## 3.1.1) Create your index!

In [9]:
def preprocessing_text(df):
    #remove upper cases
    df=df.lower()
    #replacing new line sign '\n' with a whitespace ' '    
    df=df.replace('\\n',' ')

    #removing stop words and punctuation
    stop_words = set(stopwords.words('english')) 

    #for removing punctuations
    tokenizer = RegexpTokenizer(r'\w+')
    
    #to tokenize the string
    word_tokens = tokenizer.tokenize(df) 
    
    #for removing non-english words
    eng_dic = enchant.Dict("en_US")
    
    #stemming
    ps = PorterStemmer()
    filtered_words = [ps.stem(w) for w in word_tokens if not w in stop_words] #and eng_dic.check(w)

    return filtered_words

In [41]:
#Building a vocabulary

#set for vocabulary (values of the set will be the keys fo vocabulary_dict)
vocabulary_lst=[]
#building a dictionary which will be used for making an inverted index
doc_vocabs=defaultdict(list)

for i in airbnb_data.index:
    #take one file
    df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title'],encoding='ISO-8859-1')
    #preprocessing 
    df=df.description[0]+' '+df.title[0]
    filtered_words=preprocessing_text(df)
    temp_vocabulary_set=set()
    for word in filtered_words:
        temp_vocabulary_set.add(word)
    vocabulary_lst.append(temp_vocabulary_set)
    doc_vocabs[i]=list(temp_vocabulary_set)

In [42]:
vocabulary_set=set.union(*vocabulary_lst)
len(vocabulary_set) #11717

11717

In [43]:
#mapping words into integers
vocabulary={}
for k,v in enumerate(vocabulary_set):
    #just for testing
    #vocabulary[v]='id'+str(k)
    vocabulary[v]= k

In [44]:
len(vocabulary)# before with non english 11717

11717

In [45]:
#Hint: Since you do not want to compute the inverted 
#index every time you use the Search Engine, 
#it is worth to think to store it in a separate file and load it in memory when needed.

# Save a dictionary into a pickle file.

pickle.dump(doc_vocabs, open("doc_vocabs.p", "wb"))  # save it into a file named save.p

pickle.dump(vocabulary, open("vocabulary.p", "wb"))  # save it into a file named save.p

# Load the dictionary back from the pickle file.



def save_vocabulary(vocabulary,file_name): 
    """
    method that converts vocabulary into a dataframe and saves it into a csv file
    
    input: vocabulary(dictionary, key='term',value='term_id')
    """
    vocabulary_dataframe=pd.DataFrame()
    vocabulary_dataframe['word']=vocabulary.keys()
    vocabulary_dataframe.to_csv(str(file_name)+'.csv')
    del vocabulary_dataframe

save_vocabulary(vocabulary,'vocabulary')

# for future usage it can be just imported 
vocabulary=pd.read_csv('vocabulary.csv',encoding='ISO-8859-1',index_col=[0])
vocabulary.head()

In [11]:
doc_vocabs = pickle.load(open("doc_vocabs.p", "rb"))
vocabulary = pickle.load(open("vocabulary.p", "rb"))

# Compute an inverted index

In [46]:
def compute_inverted_idx(doc_vocabs,vocabulary):
    """
    method that computes an inverted index
    
    input: doc_vocabs(dictionary), vocabulary(dictionary of all unique words, key=term, value=term_id)
    output: inverted_idx(dictionary, key=term_id, value=list of document_ids) 
    """
    #initialize defaultdict for making an inverted index
    inverted_idx = defaultdict(list)
    #in every document look for every word and assign document id to the words which belong to it
    for idx in doc_vocabs.keys():
        for word in doc_vocabs[idx]:
            inverted_idx[vocabulary[word]].append(idx)
    return inverted_idx

In [47]:
#compute an inverted index
inverted_idx=compute_inverted_idx(doc_vocabs,vocabulary)

In [48]:
#Hint: Since you do not want to compute the inverted 
#index every time you use the Search Engine, 
#it is worth to think to store it in a separate file and load it in memory when needed.

# Save a dictionary into a pickle file.


pickle.dump(inverted_idx, open("inverted_idx.p", "wb"))  # save it into a file named save.p

# Load the dictionary back from the pickle file.



In [12]:
inverted_idx = pickle.load(open("inverted_idx.p", "rb"))

In [50]:
#for example number of documents containing word whose id is 11010
len(inverted_idx[11010])

1

In [51]:
len(inverted_idx)

11717

In [52]:
type(inverted_idx)

collections.defaultdict

In [53]:
for k in vocabulary.keys():
    if vocabulary[k]==11010:
        print(k)

rainforest


# 3.1.2) Execute the query

In [60]:
def finalize_output(result_set):
    df=pd.DataFrame()
    for i,val in enumerate(result_set):
        pd.set_option('display.max_colwidth', -1)
        df=df.append(pd.read_csv('data/doc_'+str(val)+'.tsv',sep='\t',usecols=['description','title','city','url']
                                 ,encoding='ISO-8859-1'))
        df.reset_index().drop('index',axis=1,inplace=True)
    return df#[['title','description','city','url']]

In [61]:
def search_engine():
    user_query=str(input())
    #input()

    user_query=preprocessing_text(user_query)

    list_term_idx=[]
    result_set=[]
    for word in user_query:
        #if word exist in the vocabulary
        if word in vocabulary.keys():
            list_term_idx.append(set(inverted_idx[vocabulary[word]]))
        else:
            list_term_idx.append({'x'})
            break
    result_set=list(set.intersection(*list_term_idx))
    if 'x' in result_set or not result_set:
        result_set='No results! Try again!'
        return result_set
        
    print(result_set)
    result_set=finalize_output(result_set)
    return result_set

In [62]:
vocabulary['bedroom']

286

In [63]:
vocabulary['dalla']

181

In [64]:
inverted_idx[vocabulary['bedroom']]

[0,
 1,
 3,
 7,
 8,
 10,
 15,
 16,
 18,
 20,
 21,
 23,
 25,
 33,
 44,
 47,
 48,
 52,
 55,
 57,
 58,
 64,
 66,
 68,
 69,
 70,
 74,
 75,
 76,
 80,
 85,
 91,
 97,
 98,
 109,
 114,
 118,
 119,
 126,
 129,
 134,
 138,
 142,
 145,
 150,
 151,
 153,
 159,
 160,
 163,
 164,
 168,
 169,
 172,
 173,
 175,
 176,
 178,
 179,
 182,
 184,
 187,
 190,
 194,
 195,
 200,
 206,
 209,
 211,
 213,
 220,
 223,
 231,
 232,
 233,
 235,
 237,
 239,
 240,
 249,
 251,
 252,
 253,
 258,
 259,
 264,
 275,
 277,
 280,
 281,
 282,
 284,
 288,
 294,
 297,
 301,
 307,
 308,
 309,
 317,
 318,
 325,
 327,
 329,
 332,
 335,
 339,
 342,
 344,
 347,
 349,
 363,
 368,
 373,
 378,
 382,
 384,
 386,
 391,
 392,
 393,
 397,
 407,
 409,
 411,
 416,
 418,
 423,
 428,
 430,
 433,
 438,
 442,
 445,
 447,
 448,
 451,
 454,
 455,
 461,
 463,
 468,
 470,
 472,
 474,
 477,
 478,
 480,
 489,
 493,
 498,
 501,
 506,
 509,
 510,
 512,
 515,
 516,
 522,
 524,
 527,
 528,
 533,
 534,
 536,
 541,
 544,
 545,
 547,
 553,
 554,
 557,
 559,
 

In [65]:
search_engine()

sun dallas bedroom
[9058, 9988, 14695, 8140, 15438]


,city,description,title,url
0,Dallas,"Spacious Bedroom w Queen-size bed, sun room, and private bathroom in a century-old house near the center of Dallas. 4 miles from Convention Center and beyond via free Dlink, or buses &amp; light rail. 3 Blocks from Bishop Arts District. HDTV/Internet",Vintage House Near Center of Dallas,https://www.airbnb.com/rooms/1488809?location=Cedar%20Hill%2C%20TX
0,Dallas,"Spacious Bedroom w Queen-size bed, sun room, and private bathroom in a century-old house near the center of Dallas. 4 miles from Convention Center and beyond via free Dlink, or buses &amp; light rail. 3 Blocks from Bishop Arts District. HDTV/Internet",Vintage House Near Center of Dallas,https://www.airbnb.com/rooms/1488809?location=Brazos%20River%2C%20TX
0,Dallas,"Spacious Bedroom with Full size Bed and ample closet space, Nice sun facing windows, Located in a Quiet and Serene Neighborhood. It is great location within 3 minutes drive of University of Texas at Dallas, easily accessible from DFW and Lovefield Airports.\n\nWithin a minutes' walk you will find everything you need; exotic restaurants and bars: Greek,Mexican,Chinese, Asian, Indian. Retail outlets: Target , Sprouts, Natural Groceries. Beauty Salons, 24hr Gym, Massage spots,banks and much more","Spacious, and Comfortable stay in Dallas",https://www.airbnb.com/rooms/14641082?location=Addison%2C%20TX
0,Dallas,"Spacious Bedroom w Queen-size bed, sun room, and private bathroom in a century-old house near the center of Dallas. 4 miles from Convention Center and beyond via free Dlink, or buses &amp; light rail. 3 Blocks from Bishop Arts District. HDTV/Internet",Vintage House Near Center of Dallas,https://www.airbnb.com/rooms/1488809?location=Carrollton%2C%20TX
0,Dallas,"Spacious Bedroom w Queen-size bed, sun room, and private bathroom in a century-old house near the center of Dallas. 4 miles from Convention Center and beyond via free Dlink, or buses &amp; light rail. 3 Blocks from Bishop Arts District. HDTV/Internet",Vintage House Near Center of Dallas,https://www.airbnb.com/rooms/1488809?location=Arlington%2C%20TX


# 3.2) Conjunctive query & Ranking score 

## 3.2.1) Inverted index 

### Calculation of tf-idf values

In [74]:
# First way
#TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it)

def calculate_tf_idf(airbnb_data):
    tf_idf_dic=dict()
    total_num_docs=airbnb_data.shape[0]
    result_df=pd.DataFrame()
    for i in airbnb_data.index:
        #take one file
        df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title'],encoding='ISO-8859-1')
        #preprocessing 
        df=df.description[0]+' '+df.title[0] 
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs/len(inverted_idx[vocabulary[x]])))
        result_df=pd.concat([pd.Series(idf_series.values),pd.Series(tf_series.values),pd.Series(idf_calc.values)],axis=1)#.reset_index()
        result_df['tf_idf']=result_df[1]*result_df[2]

        for idx in range(result_df.shape[0]):
            tf_idf_dic[result_df[0][idx],i]=result_df['tf_idf'][idx]
    return tf_idf_dic        

In [75]:
# Second way--to check if it is the same like the 1st-for double checking the results
def calculate_tf_idf2(airbnb_data):
    idf_dic2={}
    tf_dic2={}
    proba={}
    for i in airbnb_data.index:
        #take one file
        df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title'],encoding='ISO-8859-1')
        #preprocessing 
        df=df.description[0]+' '+df.title[0] 
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
       
        for idx in range(len(tf_series)):
            idf_dic2[idf_series[idx],i]=idf_calc[idx] 
        for index,value in tf_series.iteritems():
            tf_dic2[index,i]=value
        for k in tf_dic2.keys():
            proba[k]=tf_dic2[k]*idf_dic2[k]
    return proba        

In [76]:
def compute_inverted_idx2(inverted_idx,vocabulary,tf_idf_dic):
    """
    method that computes the second inverted index
    
    input:  
    output: inverted_idx2(dictionary, key=term_id, value=list of tuples (document_id,tf_idf value) )
    """
    inverted_idx2=defaultdict(list)
    for term_id in inverted_idx.keys():
        for k,v in vocabulary.items():#k->term, v->term_id
            if v==term_id:
                term=k
        for doc_id in inverted_idx[term_id]:
            inverted_idx2[term_id].append((doc_id,tf_idf_dic[term,doc_id]))
    return inverted_idx2

In [77]:
tf_idf_dic=calculate_tf_idf(airbnb_data)

In [78]:
inverted_idx2=compute_inverted_idx2(inverted_idx,vocabulary,tf_idf_dic)

In [80]:
inverted_idx2[0]

[(16705, 0.2582214049509666)]

In [79]:
pickle.dump(inverted_idx2, open("inverted_idx2.p", "wb"))  # save it into a file 

## 3.2.2) Execute the query

In [1]:
def search_engine(k):
    user_query=str(input())
    user_query=preprocessing_text(user_query)
    user_query_tfidf=np.ones(len(user_query))

    list_term_idx=[]
    #list of dataframes
    list_tf_idf=[]

    result_set=[]

    result_tf_idf_dic=defaultdict(list)
    for word in user_query:
        #if word exist in the vocabulary
        if word in vocabulary.keys():
            list_term_idx.append(set(inverted_idx[vocabulary[word]]))
            list_tf_idf.append((inverted_idx2[vocabulary[word]]))#[:,1])
            #result_tf_idf_dic
        else:
            list_term_idx.append({'x'})
            break
    result_set=list(set.intersection(*list_term_idx))
    if 'x' in result_set or not result_set:
        result_set='No results! Try again!'

    tf_idf_dic=defaultdict(list)

    for tf_idf_1doc in list_tf_idf:
        for tuple_pair in tf_idf_1doc:
            if tuple_pair[0] in result_set:
                tf_idf_dic[tuple_pair[0]].append(tuple_pair[1])

    print(result_set)
    result_set=finalize_output(result_set,user_query_tfidf,tf_idf_dic,k)
    return result_set

In [2]:
def cosine_sim_tuples(user_query_tfidf,tf_idf_dic):
    cosine_sim_lst_tuples=[]
    for key,value in tf_idf_dic.items():
        tf_idf_val=cosine_similarity([user_query_tfidf],[value])[0][0]
        cosine_sim_lst_tuples.append((tf_idf_val,key))
    return cosine_sim_lst_tuples

In [3]:
def heapify_tuples(cosine_sim_lst_tuples,k):
    heap = []
    for item in cosine_sim_lst_tuples:
         heappush(heap, item)
    return wanted_doc(nlargest(k,heap))

In [4]:
def wanted_doc(heap_k_docs):
    wanted_doc_ids={}
    for tup in (heap_k_docs):
        wanted_doc_ids[tup[1]]=round(tup[0],2)
    return wanted_doc_ids

In [22]:
def finalize_output(result_set,user_query_tfidf,tf_idf_dic,k):
    cosine_sim_lst_tuples=cosine_sim_tuples(user_query_tfidf,tf_idf_dic)
    wanted_doc_ids=heapify_tuples(cosine_sim_lst_tuples,k)
    result_set=wanted_doc_ids.keys()
    df=pd.DataFrame()

    for i,val in enumerate(result_set):
        pd.set_option('display.max_colwidth', -1)
        df=df.append(pd.read_csv('data/doc_'+str(val)+'.tsv',sep='\t',usecols=['description','title','city','url']
                                 ,encoding='ISO-8859-1'))
        df.reset_index().drop('index',axis=1)
    df['similarity']=wanted_doc_ids.values()
    df=df[['title','description','city','url','similarity']]
    return df

In [23]:
def cosine_distance(a, b):
    cosine_similarity=(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))
    return 1 - cosine_similarity

In [24]:
#dallas private bathroom sun

In [25]:
inverted_idx2 = pickle.load(open("inverted_idx2.p", "rb"))

In [26]:
search_engine(5)#dallas sun private room

sun private room 
[17539, 9988, 16391, 17416, 2569, 269, 13205, 2847, 8997, 3367, 8872, 8747, 15918, 6705, 4405, 7102, 12867, 8140, 13773, 15438, 5454, 6875, 11362, 9058, 2151, 872, 1899, 13047, 18163, 17271, 12024]


,title,description,city,url,similarity
0,Spacious Private guest room w/sunroom - woman only,"3 minute drive to Mopac and 15 minutes to Downtown, close to Bike Trails and Lady Bird Johnson's Bird Park. My place is a 5 minute walk to boutique-style Escarpment Village Shopping with grocery, a Starbucks, higher end dining, or grab an ice cream or burger.\n\nYoull enjoy the coziness of the private master bedroom, private bath, and French door entry to the sun room, with view of beautiful trees in backyard. My place is great for couples, solo adventurers, or business travelers.",Austin,https://www.airbnb.com/rooms/18868758?location=Bee%20Cave%2C%20TX,0.97
0,Private Room in Warm Boutique Home,"Congratulations on finding the perfect place to stay. We have a Private room with a ceiling fan, full size closet, wireless access, access to swimming pool, washer and dryer, private patio for relaxing in the shade or laying out in the sun, close to Hwy (PHONE NUMBER HIDDEN), and the Dell Campus. austin is 15 minutes away and COTA is 30 minutes.",Round Rock,https://www.airbnb.com/rooms/7672843?location=Coupland%2C%20TX,0.97
0,"Cozy/Private Room, Minutes to Downtown!","Cozy private room, very close to the airport and 7 miles from downtown Austin. The room is simple but equipped with everything you would need for a comfortable stay. Double bed, two bed side tables, full closet to tuck away your belongings in. The full bath is shared with another air bnb listing. Bathroom has toiletries and hairdryer. Feel free to use our living room, dining area and kitchen. We also have a nice back patio great for soaking up sun or night drinking.",Austin,https://www.airbnb.com/rooms/17423956?location=Austin%2C%20TX,0.97
0,"Comfy, Quiet Private Room with Late Check-Out","Our place is very quiet and clean. Comfy queen sized bed and black-out curtain on the window so you can sleep in without with sun waking you.\n\nWe are off the beaten path, in a quiet neighborhood east of Austin. A short drive 15-30 minutes (depending on where you are headed and when) to a ton of great Austin sites and sounds. \n\nOur place is perfect if you are looking for a comfortable, clean, quiet private room.",Austin,https://www.airbnb.com/rooms/16827574?location=Bastrop%20County%2C%20TX,0.96
0,"Comfy, Quiet Private Room with Late Check-Out","Our place is very quiet and clean. Comfy queen sized bed and black-out curtain on the window so you can sleep in without with sun waking you.\n\nWe are off the beaten path, in a quiet neighborhood east of Austin. A short drive 15-30 minutes (depending on where you are headed and when) to a ton of great Austin sites and sounds. \n\nOur place is perfect if you are looking for a comfortable, clean, quiet private room.",Austin,https://www.airbnb.com/rooms/16827574?location=Cedar%20Creek%2C%20TX,0.96
